# Set up env

In [1]:
%%sh
# current_path=$(pwd)
# parent_dir=$(dirname "$current_dir")
export PROJECT_HOME="/home/ivan/Uforse/university_crawl"
echo $PROJECT_HOME
if [ -f "${PROJECT_HOME}/build" ]; then
    echo "Found build in PROJECT_HOME. Sourcing now..."
    . "${PROJECT_HOME}/build"
    echo "build executed successfully."
else
    echo "Error: build not found in PROJECT_HOME."
fi

/home/ivan/Uforse/university_crawl
Found build in PROJECT_HOME. Sourcing now...
Obtaining file:///home/ivan/Uforse/university_crawl/university_info_generator
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: university-info-generator
    Found existing installation: university-info-generator 0.0.3
    Uninstalling university-info-generator-0.0.3:
      Successfully uninstalled university-info-generator-0.0.3
  Running setup.py develop for university-info-generator
Directory already exists: /home/ivan/Uforse/university_crawl/cache_repo
Environment variables set:
PROJECT_HOME=/home/ivan/Uforse/university_crawl
PYTHONPATH=/home/ivan/Uforse/university_crawl:
API_KEY_PATH=/home/ivan/Uforse/university_crawl
CACHE_REPO_PATH=/home/ivan/Uforse/university_crawl/cache_repo
LANGCHAIN_API_KEY=ls__3780df60c0ef4fd2b2414f032feb31ce
Environment set up for uni_info_generator
build executed successfully.


In [2]:
import os
import sys
from pprint import pprint

#'/home/ivan/Uforse/university_crawl/fetch_logic'
current_path = os.path.abspath('./')

#'/home/ivan/Uforse/university_crawl'
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import sys
print(sys.executable)
sys.path.append('/home/ivan/Uforse/university_crawl')

/home/ivan/anaconda3/envs/gpu/bin/python


# Import

In [3]:
import pandas as pd
from pprint import pprint
from typing import Dict
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import concurrent.futures

from university_info_generator import config
from university_info_generator import UniversityInfoGenerator
from university_info_generator import UniversityProgramScraper
from university_info_generator import Program


# Logic

In [4]:
scraper = UniversityProgramScraper()
canada_df = pd.read_csv(f"{scraper.data_repo_path}/canada_dataset/all_universities_canada.csv")
qs_df = pd.read_csv(f"{scraper.data_repo_path}/qs_ranking_2024.csv")

In [5]:
# # set unique id
# lst = []
# Program.set_program_id(0)
# for i in range(10):
#     lst.append(Program())
# for pro in lst:
#     print(pro.to_dict_en())

## canada

In [6]:
matched_df = UniversityProgramScraper.join_df(canada_df, qs_df)

In [7]:
scraper = UniversityProgramScraper(need_selenium = False)


In [8]:
canada_info_df = pd.read_csv(f"{scraper.data_repo_path}/canada_dataset/canada_info.csv", index_col="id_")


## scarp on one link

In [9]:
uwaterloo_program_lst = scraper.get_program_link_lst(link = "https://www.topuniversities.com/universities/university-waterloo")
print(uwaterloo_program_lst)

{'bachelor': {'Arts and Humanities': {'Architectural Engineering (BAS)': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/architectural-engineering-bas', 'Architecture': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/architecture', 'Classical Studies': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/classical-studies', 'English': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/english', 'Fine Arts': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/fine-arts', 'French': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/french', 'Geography and Aviation': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/geography-aviation', 'Geography and Environmental Management': 'https://www.topuniversities.com/universities/university-waterloo/undergrad/geography-environmental-management', 'Geological Engineering 

In [10]:
import requests
from bs4 import BeautifulSoup, ResultSet, Tag

In [11]:
# break

In [12]:
# # fetch one university
# Program.set_program_id(0)


# def fetch_from_program_lst(program_lst: Dict[str, Dict[str, Dict[str, str]]], university_name: str):
#     result_dict = {}
#     key_set = Program.valid_keys
#     key_set.difference_update(["program_id"])

#     for study_degree in program_lst:
#         for qs_general_program in program_lst[study_degree]:
#             for program_name in tqdm(program_lst[study_degree][qs_general_program]):
#                 # print(program_name)
#                 # program =Program(university_name=university_name)
#                 program = Program()
#                 for key in key_set:
#                     program.update({key: ""})
#                 link = program_lst[study_degree][qs_general_program][program_name]
#                 program.update({"university_name": university_name, "qs_program_link": link, "qs_general_program": qs_general_program})
#                 # program = Program(university_name=university_name)
#                 program = scraper.extract_program_by_link(link, program=program)
#                 result_dict[f"{study_degree} - {program_name}"] = program.to_dict_en()
#     return result_dict


In [25]:
def scrape_program_data(link, university_name, qs_general_program, key_set):
    program = Program()
    for key in key_set:
        program.update({key: ""})
    program.update({"university_name": university_name, "qs_program_link": link, "qs_general_program": qs_general_program})
    program = UniversityProgramScraper.extract_program_by_link(link, program=program)
    return program.to_dict_en()

def fetch_from_program_lst(program_lst: Dict[str, Dict[str, Dict[str, str]]], university_name: str):
    result_dict = {}
    key_set = Program.valid_keys
    key_set.difference_update(["program_id"])

    total_programs = sum(len(program_lst[study_degree][qs_general_program]) for study_degree in program_lst for qs_general_program in program_lst[study_degree])

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_program = {}

        for study_degree in program_lst:
            for qs_general_program in program_lst[study_degree]:
                for program_name in program_lst[study_degree][qs_general_program]:
                    link = program_lst[study_degree][qs_general_program][program_name]
                    future = executor.submit(scrape_program_data, link, university_name, qs_general_program, key_set)
                    future_to_program[future] = f"{study_degree} - {program_name}"
        with tqdm(total=total_programs, desc="Scraping Programs") as pbar:
            for future in concurrent.futures.as_completed(future_to_program):
                program_name = future_to_program[future]
                try:
                    result_dict[program_name] = future.result()
                except Exception as exc:
                    print(f'Program {program_name} generated an exception: {exc}')
                pbar.update(1)
    return result_dict

In [26]:
result = fetch_from_program_lst(uwaterloo_program_lst, "Universty of Waterloo")

Scraping Programs:   0%|          | 0/208 [00:00<?, ?it/s]

Scraping Programs: 100%|██████████| 208/208 [00:44<00:00,  4.65it/s]


In [27]:
scraper.quit_driver()

In [28]:
# df = pd.DataFrame(result.items(), columns=["index"] + list(Program.valid_keys))
# df = pd.DataFrame.from_dict(result)
# df = df[["index"] + list(Program.valid_keys)]
df = pd.DataFrame(result).T

In [17]:
df.head()

""


In [29]:
df.to_csv("/home/ivan/Uforse/university_crawl/university_info_generator/fetch_logic/fetch_data/programs/canada/uwaterloo_programs.csv", encoding="utf-8")

## usa


In [19]:
us_df = pd.read_csv(f"{scraper.data_repo_path}/usa_dataset/all_universities_usa.csv")
matched_usa_df = UniversityProgramScraper.join_df(us_df, qs_df, save_path=f"{config.PROJECT_HOME}/university_info_generator/data/usa_dataset/qs_usa_uni_matched.csv")